# Comparative analysis of brain organisation in two brain regions

In this tutorial, we use `siibra` to analyze multimodal aspects of two different brain regions. We select a region from the Broca region in the inferior frontal gyrus, IFG 44, involved in language prociessing, and a region from the visual system in the occipital cortex, V1.

### Load packages

We start by importing the siibra package and libraries for visualization.

In [ ]:
import siibra
assert siibra.__version__ >= "1.0.1"
import matplotlib.pyplot as plt
from nilearn import plotting
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('dark')

### Instantiate parcellation and reference space from the human brain atlas

In [ ]:
jubrain = siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_ATLAS_V3_0_3
pmaps = jubrain.get_map(space='mni152', maptype='statistical')

### Instantiate the two regions of interest

In [ ]:
specs = ['ifg 44 left', 'hoc1 left']
regions = [jubrain.get_region(spec) for spec in specs]
for r in regions:
    plotting.plot_glass_brain(
        pmaps.fetch(region=r), cmap='viridis', draw_cross=False, colorbar=False, annotate=False, symmetric_cbar=True
    )
    plt.savefig(f"{r.key}.png")

### Compare average receptor and cortical cell densities

In [ ]:
receptors = ['M1', 'M2', 'M3', '5-HT1A', '5-HT2', 'D1']  # selection of monogenetic receptors
genes=[
    siibra.vocabularies.GENE_NAMES.CHRM1,
    siibra.vocabularies.GENE_NAMES.CHRM2,
    siibra.vocabularies.GENE_NAMES.CHRM3,
    siibra.vocabularies.GENE_NAMES.HTR1A,
    siibra.vocabularies.GENE_NAMES.HTR2A,
    siibra.vocabularies.GENE_NAMES.DRD1
]
modalities = [
    (siibra.features.molecular.ReceptorDensityFingerprint, {}, {"rot": 90}),
    (siibra.features.cellular.LayerwiseCellDensity, {}, {"rot":0}),
    (siibra.features.molecular.GeneExpressions, {'gene': genes}, {"rot":90})
]
fig, axs = plt.subplots(len(modalities)+1, len(regions), figsize=(4*len(regions),11), sharey='row')
ymax = [1000, 150, None]

for i, region in enumerate(regions): 
    axs[0, i].imshow(plt.imread(f"{region.key}.png"))
    axs[0, i].set_title(f'{region.name.replace("Area ", "")}')
    axs[0, i].axis('off')
    for j, (modality, kwargs, plotargs) in enumerate(modalities):
        fts = siibra.features.get(region, modality, **kwargs)
        assert len(fts) > 0
        if len(fts) > 1:
            print(f"More than one feature found for {modality}, {region.name}")
        f = fts[0]
        if modality == siibra.features.molecular.ReceptorDensityFingerprint:
            fcopy = f
            fcopy._data_cached = f.data.loc[receptors]
        f.plot(ax = axs[j+1, i], **plotargs)
        if modality == siibra.features.molecular.ReceptorDensityFingerprint:
            # add neurotransmitter names to  receptor names in xtick labels
            transmitters = [re.sub(r'(^.*\()|(\))', '', n) for n in f.neurotransmitters]
            axs[j+1, i].set_xticklabels([f"{r}\n({n})" for r, n in zip(f.receptors, transmitters)])
        if ymax[j] is not None:
            axs[j+1, i].set_ylim(0, ymax[j])
        if 'std' in axs[j+1, i].yaxis.get_label_text():
            axs[j+1, i].set_ylabel(axs[j+1, i].yaxis.get_label_text().replace('std', 'std\n'))
        axs[j+1, i].set_title(f'{fts[0].modality}')
fig.suptitle('')
fig.tight_layout()

In [ ]:
fts = siibra.features.get(regions[0], siibra.features.connectivity.StreamlineCounts)
conn = fts[0]

# aggregate connectivity profiles for first region across subjects
D1 = (
    pd.concat([c.get_profile(regions[0]).data for c in conn], axis=1)
    .agg(['mean', 'std'], axis=1)
    .sort_values(by='mean', ascending=False)
)

# aggregate connectivity profiles for second region across subjects
D2 = (
    pd.concat([c.get_profile(regions[1]).data for c in conn], axis=1)
    .agg(['mean', 'std'], axis=1)
    .sort_values(by='mean', ascending=False)
)

In [ ]:
# plot both average connectivity profiles to target regions
def shorten_name(n):
    return n.replace('Area ', '').replace(' (GapMap)', '').replace('left', 'L').replace('right', 'R')
    
fig, (a1, a2) = plt.subplots(1, 2, sharey=True, figsize=(3*len(regions), 3.5))
kwargs = {'kind':'bar', 'width': 0.85, 'logy':True}
D1.iloc[:15]['mean'].plot(**kwargs, yerr=D1.iloc[:15]['std'], ax=a1, ylabel=shorten_name(regions[0].name))
D2.iloc[:15]['mean'].plot(**kwargs, yerr=D2.iloc[:15]['std'], ax=a2, ylabel=shorten_name(regions[1].name))
a1.set_ylabel('Streamline counts')
a1.xaxis.set_ticklabels([shorten_name(t.get_text()) for t in a1.xaxis.get_majorticklabels()])
a2.xaxis.set_ticklabels([shorten_name(t.get_text()) for t in a2.xaxis.get_majorticklabels()])
plt.tight_layout()

### Retrieve cortical patches

In [ ]:
layers = siibra.parcellations.get('cortical layers')
layermap = layers.get_map(space='bigbrain')
patches = {}
import numpy as np

for r in regions:
    # find 1 micron sections intersecting the region
    pmap_volume = siibra.volumes.from_nifti(pmaps.fetch(region=r), space='mni152', name=r.name)
    sections = siibra.features.get(pmap_volume.get_boundingbox(), siibra.features.cellular.CellbodyStainedSection)
    section = sections[len(sections)//2]

    # intersect layer IV surface with the selected section
    imgplane = siibra.experimental.Plane3D.from_image(section)
    hemisphere = 'left' if 'left' in r.name else 'right'
    l4layer = layers.get_region(f"4 {hemisphere}")
    l4_contours = imgplane.intersect_mesh(layermap.fetch(region=l4layer, format='mesh'))
    
    # score all contour points with the region probabilities to find the most likely one
    l4points = siibra.PointCloud(
        [p for contour in l4_contours for p in contour], 
        space='bigbrain'
    )
    probs = pmap_volume.evaluate_points(l4points)
    l4point = l4points[probs.argmax()]
    
    # find the closest cortical profile to this point
    profile = siibra.experimental.CorticalProfileSampler().query(l4point)
    
    # extract the patch in the image plane showing the projection of this profile
    patch_canvas = imgplane.get_enclosing_patch(profile)
    i = np.argmin(
        np.linalg.norm((patch_canvas.corners.coordinates - profile[0].coordinate), axis=1)
    )
    if i in [0, 3]:
        patch_canvas.flip()
    patches[r] = patch_canvas.extract_volume(section, resolution_mm=0.01)

In [ ]:
# display the two extracted patches
fig, axs = plt.subplots(1, 2)
for ax, patch, region in zip(axs, patches, regions):
    
    # access the underlying image data
    patchimg = patches[patch].fetch()
    phys2vox = np.linalg.inv(patchimg.affine)
    patchdata = patchimg.get_fdata().squeeze()
    
    ax.set_title(region.name)

    # plot the pure image array
    ax.imshow(patchdata, cmap='gray', vmin=0, vmax=2**16)
    ax.axis('off')

plt.tight_layout()